<font size="+3"><b><center>DP IMP OV Data Pipeline</center></b></font>

### Imports

In [1]:
%load_ext autoreload
%autoreload 2
import os
import datetime
import pandas as pd
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 100)
from pyspark.sql import functions as F

from crf0a_app.configuration import spark_config
from crf0a_app.utils import system

### Spark session

In [2]:
spark_context, spark_session = spark_config.get_spark(
    app_name="[app00] Test_Read_Write_Data",
    driver_cores=1,
    driver_mem="4g",
    max_executors=8,
    executor_cores=4,
    executor_mem="4g"
)

### HDFS

#### Read

In [6]:
# Data filepath
cycle = '202201'
flow_filepath = "/user/sd01865/crf0a/data/iCube/PROREV_%sIMP_OV.csv" %cycle
print(flow_filepath)

/user/sd01865/crf0a/data/iCube/PROREV_202201IMP_OV.csv


In [20]:
# Read data from HDFS
df_flow = spark_session.read.options(header='True', inferSchema='True', delimiter=';').csv(flow_filepath)
df_flow.printSchema()

root
 |-- Marque: string (nullable = true)
 |-- Pays: string (nullable = true)
 |-- Libelle pays: string (nullable = true)
 |-- Code version: string (nullable = true)
 |-- Date: integer (nullable = true)
 |-- Volume 01: integer (nullable = true)
 |-- Volume 02: integer (nullable = true)
 |-- Volume 03: integer (nullable = true)
 |-- Volume 04: integer (nullable = true)
 |-- Volume 05: integer (nullable = true)
 |-- Volume 06: integer (nullable = true)
 |-- Volume 07: integer (nullable = true)
 |-- Volume 08: integer (nullable = true)
 |-- Volume 09: integer (nullable = true)
 |-- Volume 10: integer (nullable = true)
 |-- Volume 11: integer (nullable = true)
 |-- Volume 12: integer (nullable = true)



In [21]:
df_flow.persist()

DataFrame[Marque: string, Pays: string, Libelle pays: string, Code version: string, Date: int, Volume 01: int, Volume 02: int, Volume 03: int, Volume 04: int, Volume 05: int, Volume 06: int, Volume 07: int, Volume 08: int, Volume 09: int, Volume 10: int, Volume 11: int, Volume 12: int]

In [31]:
df_flow.limit(5).toPandas()
df_flow.printSchema()

root
 |-- Marque: string (nullable = true)
 |-- Pays: string (nullable = true)
 |-- Libelle pays: string (nullable = true)
 |-- Code version: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Volume 01: integer (nullable = true)
 |-- Volume 02: integer (nullable = true)
 |-- Volume 03: integer (nullable = true)
 |-- Volume 04: integer (nullable = true)
 |-- Volume 05: integer (nullable = true)
 |-- Volume 06: integer (nullable = true)
 |-- Volume 07: integer (nullable = true)
 |-- Volume 08: integer (nullable = true)
 |-- Volume 09: integer (nullable = true)
 |-- Volume 10: integer (nullable = true)
 |-- Volume 11: integer (nullable = true)
 |-- Volume 12: integer (nullable = true)



In [23]:
df_flow.count()

9860

### Transformations

In [29]:
#change the data type of the Date column
df_flow = df_flow.withColumn("Date", F.to_date(F.col("Date").cast("string"), \
    'yyyyMM'))

In [30]:
df_flow.limit(5).toPandas()

,Marque,Pays,Libelle pays,Code version,Date,Volume 01,Volume 02,Volume 03,Volume 04,Volume 05,Volume 06,Volume 07,Volume 08,Volume 09,Volume 10,Volume 11,Volume 12
0,G,BG00010,BULGARIA OV,1GJOA5HMT1B0A0D4,2022-01-01,0,0,0,0,5,2,2,2,2,2,2,2
1,G,BG00010,BULGARIA OV,1GJOA5HPXKB0A0D4,2022-01-01,0,0,0,0,3,0,0,0,0,0,0,0
2,G,BG00010,BULGARIA OV,1GJOA5TESFB0A0D4,2022-01-01,0,0,0,0,1,0,0,0,0,0,0,0
3,G,BG00010,BULGARIA OV,1GJOA5TNP1B0A0D4,2022-01-01,0,0,0,0,12,0,0,0,0,0,0,0
4,G,BG00010,BULGARIA OV,1GJOA5TPXKB0A0D4,2022-01-01,0,0,0,0,10,0,0,0,0,0,0,0


In [32]:
#get the date value we will need to change the volume columns names
the_date = df_flow.collect()[0][4]
print(the_date)

2022-01-01


In [41]:
#Rename the Valume colums with the date of the first of every month starting from the_date 
df_flow = df_flow.withColumnRenamed("Volume 01",str(the_date)) \
    .withColumnRenamed("Volume 02",str((the_date + datetime.timedelta(days=32)).replace(day=1))) \
    .withColumnRenamed("Volume 03",str((the_date + datetime.timedelta(days=32*2)).replace(day=1))) \
    .withColumnRenamed("Volume 04",str((the_date + datetime.timedelta(days=32*3)).replace(day=1))) \
    .withColumnRenamed("Volume 05",str((the_date + datetime.timedelta(days=32*4)).replace(day=1))) \
    .withColumnRenamed("Volume 06",str((the_date + datetime.timedelta(days=32*5)).replace(day=1))) \
    .withColumnRenamed("Volume 07",str((the_date + datetime.timedelta(days=32*6)).replace(day=1))) \
    .withColumnRenamed("Volume 08",str((the_date + datetime.timedelta(days=32*7)).replace(day=1))) \
    .withColumnRenamed("Volume 09",str((the_date + datetime.timedelta(days=32*8)).replace(day=1))) \
    .withColumnRenamed("Volume 10",str((the_date + datetime.timedelta(days=32*9)).replace(day=1))) \
    .withColumnRenamed("Volume 11",str((the_date + datetime.timedelta(days=32*10)).replace(day=1))) \
    .withColumnRenamed("Volume 12",str((the_date + datetime.timedelta(days=32*11)).replace(day=1))) \
    .withColumnRenamed("Date","Cycle")

In [55]:
pd_df=df_flow.toPandas()

In [56]:
print(pd_df.columns)

Index(['Marque', 'Pays', 'Libelle pays', 'Code version', 'Cycle', '2022-01-01',
       '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01',
       '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01',
       '2022-12-01'],
      dtype='object')


In [65]:
#pivot the table
pivoted_df = pd_df.melt(id_vars = ['Marque', 'Pays','Libelle pays','Code version','Cycle'], value_vars = ['2022-01-01',
       '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01', '2022-06-01',
       '2022-07-01', '2022-08-01', '2022-09-01', '2022-10-01', '2022-11-01',
       '2022-12-01'], var_name = 'Date', value_name = 'Value')

In [66]:
pivoted_df

,Marque,Pays,Libelle pays,Code version,Cycle,Date,Value
0,G,BG00010,BULGARIA OV,1GJOA5HMT1B0A0D4,2022-01-01,2022-01-01,0
1,G,BG00010,BULGARIA OV,1GJOA5HPXKB0A0D4,2022-01-01,2022-01-01,0
2,G,BG00010,BULGARIA OV,1GJOA5TESFB0A0D4,2022-01-01,2022-01-01,0
3,G,BG00010,BULGARIA OV,1GJOA5TNP1B0A0D4,2022-01-01,2022-01-01,0
4,G,BG00010,BULGARIA OV,1GJOA5TPXKB0A0D4,2022-01-01,2022-01-01,0
...,...,...,...,...,...,...,...
118315,G,NC00010,NEW CALEDONIA OV,1GMEAFPI2QJ0A0E0,2022-01-01,2022-12-01,0
118316,G,NC00010,NEW CALEDONIA OV,1GMEAFPI2QJ0A0E2,2022-01-01,2022-12-01,0
118317,G,NC00010,NEW CALEDONIA OV,1GJOA5HPR5J0A0D0,2022-01-01,2022-12-01,0
118318,G,NC00010,NEW CALEDONIA OV,1GJOA5HPR5J0A0D2,2022-01-01,2022-12-01,0


In [67]:
#slice the Code version values to leave only the first 4 characters
pivoted_df['Code version']=pivoted_df['Code version'].str[:4]

In [68]:
pivoted_df

,Marque,Pays,Libelle pays,Code version,Cycle,Date,Value
0,G,BG00010,BULGARIA OV,1GJO,2022-01-01,2022-01-01,0
1,G,BG00010,BULGARIA OV,1GJO,2022-01-01,2022-01-01,0
2,G,BG00010,BULGARIA OV,1GJO,2022-01-01,2022-01-01,0
3,G,BG00010,BULGARIA OV,1GJO,2022-01-01,2022-01-01,0
4,G,BG00010,BULGARIA OV,1GJO,2022-01-01,2022-01-01,0
...,...,...,...,...,...,...,...
118315,G,NC00010,NEW CALEDONIA OV,1GME,2022-01-01,2022-12-01,0
118316,G,NC00010,NEW CALEDONIA OV,1GME,2022-01-01,2022-12-01,0
118317,G,NC00010,NEW CALEDONIA OV,1GJO,2022-01-01,2022-12-01,0
118318,G,NC00010,NEW CALEDONIA OV,1GJO,2022-01-01,2022-12-01,0


In [71]:
#add the Inserted_date column
pivoted_df['INSERTED_DATE'] = datetime.date.today()

In [73]:
#add the Measure column
pivoted_df['Measure'] = 'DP IMP'

In [96]:
#Rename some columns to match the destination columns in Oracle Exadata
pivoted_df = pivoted_df.rename(columns={"Date":"MONTHYEAR"})
pivoted_df = pivoted_df.rename(columns={"Code version":"FAMILLE"})
pivoted_df = pivoted_df.rename(columns={"Libelle pays":"LIBELLE_PAYS"})
pivoted_df

,Marque,Pays,LIBELLE_PAYS,FAMILLE,MONTHYEAR,Measure,Value,Cycle,INSERTED_DATE
0,G,BG00010,BULGARIA OV,1GJO,2022-01-01,DP IMP,0,2022-01-01,2022-07-13
1,G,BG00010,BULGARIA OV,1GJO,2022-01-01,DP IMP,0,2022-01-01,2022-07-13
2,G,BG00010,BULGARIA OV,1GJO,2022-01-01,DP IMP,0,2022-01-01,2022-07-13
3,G,BG00010,BULGARIA OV,1GJO,2022-01-01,DP IMP,0,2022-01-01,2022-07-13
4,G,BG00010,BULGARIA OV,1GJO,2022-01-01,DP IMP,0,2022-01-01,2022-07-13
...,...,...,...,...,...,...,...,...,...
118315,G,NC00010,NEW CALEDONIA OV,1GME,2022-12-01,DP IMP,0,2022-01-01,2022-07-13
118316,G,NC00010,NEW CALEDONIA OV,1GME,2022-12-01,DP IMP,0,2022-01-01,2022-07-13
118317,G,NC00010,NEW CALEDONIA OV,1GJO,2022-12-01,DP IMP,0,2022-01-01,2022-07-13
118318,G,NC00010,NEW CALEDONIA OV,1GJO,2022-12-01,DP IMP,0,2022-01-01,2022-07-13


In [97]:
pivoted_df.columns

Index(['Marque', 'Pays', 'LIBELLE_PAYS', 'FAMILLE', 'MONTHYEAR', 'Measure',
       'Value', 'Cycle', 'INSERTED_DATE'],
      dtype='object')

In [99]:
#reordering the columns
cols = ('Marque', 'Pays', 'LIBELLE_PAYS', 'FAMILLE','MONTHYEAR','Measure','Value','Cycle','INSERTED_DATE')
pivoted_df = pivoted_df[list(cols)]
pivoted_df

,Marque,Pays,LIBELLE_PAYS,FAMILLE,MONTHYEAR,Measure,Value,Cycle,INSERTED_DATE
0,G,BG00010,BULGARIA OV,1GJO,2022-01-01,DP IMP,0,2022-01-01,2022-07-13
1,G,BG00010,BULGARIA OV,1GJO,2022-01-01,DP IMP,0,2022-01-01,2022-07-13
2,G,BG00010,BULGARIA OV,1GJO,2022-01-01,DP IMP,0,2022-01-01,2022-07-13
3,G,BG00010,BULGARIA OV,1GJO,2022-01-01,DP IMP,0,2022-01-01,2022-07-13
4,G,BG00010,BULGARIA OV,1GJO,2022-01-01,DP IMP,0,2022-01-01,2022-07-13
...,...,...,...,...,...,...,...,...,...
118315,G,NC00010,NEW CALEDONIA OV,1GME,2022-12-01,DP IMP,0,2022-01-01,2022-07-13
118316,G,NC00010,NEW CALEDONIA OV,1GME,2022-12-01,DP IMP,0,2022-01-01,2022-07-13
118317,G,NC00010,NEW CALEDONIA OV,1GJO,2022-12-01,DP IMP,0,2022-01-01,2022-07-13
118318,G,NC00010,NEW CALEDONIA OV,1GJO,2022-12-01,DP IMP,0,2022-01-01,2022-07-13


In [101]:
from crf0a_app.infra.oracle_database import OracleDatabase

In [102]:
#Instantiate OracleDatabase object
oracle_db = OracleDatabase(dialect="jdbc", spark_session=spark_session)

In [104]:
#Convert the pandas dataframe to pyspark dataframe
sparkDF=spark_session.createDataFrame(pivoted_df) 

#### Write

In [106]:
# Write data to Oracle
oracle_db.write_df_to_oracle(
    sparkDF,
    "BRC_SD01865.DP_IMP_OV",
    mode="append"
)